# Laboratorio: Recommendation con Algebra Lineare e NumPy

**Programmazione di Applicazioni Data Intensive**  
Laurea in Ingegneria e Scienze Informatiche  
DISI - Università di Bologna, Cesena

Proff. Gianluca Moro, Roberto Pasolini  
nome.cognome@unibo.it

## Recommendation: da strutture dati Python a matrici NumPy

- Nella scorsa esercitazione abbiamo introdotto la recommendation di prodotti
  - sappiamo quali utenti di un sito di e-commerce hanno acquistato quali prodotti
  - vogliamo fornire suggerimenti personalizzati a ciascun utente su quali ulteriori prodotti dovrebbe acquistare
- Abbiamo implementato una soluzione utilizzando le strutture dati e le funzioni standard di Python
  - dizionari, insiemi, comprehension, ...
- In questa esercitazione ripetiamo gli stessi passaggi usando array NumPy
- Vediamo così come un problema reale si possa modellare nell'ambito dell'algebra lineare e risolvere tramite semplici operazioni tra matrici

## Scaricamento File Dati

- Riutilizziamo i dati della scorsa esercitazione, scaricabili all'URL https://git.io/fhxQh
- Se state usando Binder, avete già i file nella directory corrente
- Altrimenti, eseguite la seguente cella di codice per scaricare il file ZIP dall'URL sopra ed estrarne i file

In [1]:
import os.path
if not os.path.exists("purchases_data.zip"):
    from urllib.request import urlretrieve
    urlretrieve("https://git.io/fhxQh", "purchases_data.zip")
    from zipfile import ZipFile
    with ZipFile("purchases_data.zip") as f:
        f.extractall()

## Caricamento Dati

- Ripetiamo i passaggi della scorsa esercitazione per caricare i dati dai file
  - il file `users.csv` contiene ID e nomi degli utenti analizzati: li carichiamo in un dizionario `users` che associ i nomi agli ID
  - facciamo lo stesso col file `items.csv`, che contiene ID e nomi dei prodotti: otteniamo un dizionario `items`
  - il file `purchases-2000.csv` contiene gli acquisti fino alla fine del 2000 in forma di tuple ID utente + ID prodotto: le carichiamo in un insieme `purchases_set`

In [16]:
import csv
with open("users.csv", "r") as f:
    users = {int(uid): name for uid, name in csv.reader(f, delimiter=";")}
with open("items.csv", "r") as f:
    items = {int(iid): name for iid, name in csv.reader(f, delimiter=";")}
with open("purchases-2000.csv", "r") as f:
    purchases_set = {(int(uid), int(iid)) for uid, iid in csv.reader(f, delimiter=";")}

## Algebra Lineare e NumPy

- Oggetto di studio dell'**algebra lineare** sono i vettori, le matrici e le principali operazioni tra di essi
  - con vettori e matrici si possono rappresentare dati da analizzare
  - informazioni d'interesse si possono estrarre con operazioni come il prodotto tra matrici
- **NumPy** è la libreria Python _standard de facto_ per lavorare con array multidimensionali, inclusi vettori e matrici
  - sugli array NumPy si possono compiere efficientemente operazioni elemento per elemento, aggregazioni, operazioni di algebra lineare, ecc.
- Per lavorare con NumPy lo importiamo usando l'alias convenzionale `np`

In [5]:
import sys
sys.path.append("/home/rrobby/.local/lib/python3.5/site-packages")

In [6]:
import numpy as np

## Rappresentare gli Acquisti in forma di Matrice

- L'insieme `purchases` degli acquisti degli utenti analizzati può essere rappresentato in una matrice binaria (ovvero di valori 0 e 1)
  - ogni **riga** corrisponde ad un **utente**
  - ogni **colonna** corrisponde ad un **prodotto** distinto
  - il valore della cella _i,j_ è 1 se l'utente i ha acquistato il prodotto j, 0 altrimenti
- Vediamo come costruire tale matrice

### Assegnazione di Indici a Utenti e Prodotti

- Iniziando definendo **a quali utenti e prodotti corrispondano** rispettivamente **righe e colonne** della matrice
- Costruiamo un dizionario `user_indices` che associ ad ognuno degli N ID utente un numero tra 0 e N-1
  - usiamo la funzione `enumerate` di Python per generare numeri progressivi da 0 a N-1: per ogni elemento di una collezione è restituita una tupla `(indice, elemento)`

In [8]:
user_indices = {uid: index for index, uid in enumerate(users.keys())}

- Eseguiamo un'operazione simile per ottenere un dizionario `item_indices` con la numerazione degli oggetti

In [9]:
item_indices = {iid: index for index, iid in enumerate(items.keys())}

- Abbiamo così ottenuto dizionari che mappano ID utenti e prodotti a righe e colonne della matrice
- Ad esempio, la riga corrispondente all'utente con ID 63776 è...

In [14]:
user_indices[63776]

101

### Inizializzazione della Matrice

- Per allocare la matrice degli acquisti, definiamone dapprima la _forma_, ovvero il numero di righe e di colonne
- Questi sono pari rispettivamente al numero di utenti e di prodotti, che salviamo in due variabili per comodità

In [15]:
n_users = len(users)
n_items = len(items)

- Creiamo ora la matrice `purchases` inizializzando tutti i valori a 0, ovvero senza alcun acquisto
- Usiamo per questo la funzione `zeros` passando la forma desiderata della matrice

In [20]:
purchases = np.zeros((n_users, n_items), dtype=np.int8)

### Scrittura Acquisti nella Matrice

- Per ottenere la matrice degli acquisti effettiva, impostiamo ad 1 gli elementi corrispondenti alle coppie utente-oggetto nell'insieme `purchases_set`
- Per ogni tupla, usiamo i dizionari `user_indices` e `item_indices` per individuare la posizione giusta nella matrice e impostiamo ad 1 il valore

In [21]:
for uid, iid in purchases_set:
    i = user_indices[uid]
    j = item_indices[iid]
    purchases[i, j] = 1

- Visualizziamo una porzione della matrice caricata...

In [24]:
purchases[-5:, :15]   # ultime 5 righe, prime 15 colonne

array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int8)

## Totali Acquisti

- Col metodo `sum` possiamo effettuare la somma per righe e per colonne della matrice per individuare
  - il numero totale di prodotti acquistati da ciascun utente
  - il numero totale di utenti che hanno acquistato ciascun prodotto

In [25]:
users_purchases = purchases.sum(1)   # somma tutte le colonne
items_purchases = purchases.sum(0)   # somma tutte le righe

- Possiamo verificare che ogni utente considerato abbia almeno 30 acquisti

In [26]:
users_purchases.min()

30

- Qual è il numero medio di prodotti acquistati per utente ?

In [27]:
mean_purchases = users_purchases.sum() / n_users
mean_purchases

54.39887640449438

- Qual è il numero max di prodotti acquistati da un qualche utente ?

In [28]:
users_purchases.max()

242

_**Quesito:** quanti utenti hanno acquistato il prodotto più venduto?_

## Similarità tra Utenti

- Vogliamo suggerire prodotti agli utenti in base a **cos'hanno acquistato utenti simili**
- Come determinare quanto due utenti siano "simili"?
- Possiamo contare **quanti sono i prodotti che entrambi hanno acquistato**
- Questo numero è dato dal **prodotto scalare delle righe** corrispondenti nella matrice degli acquisti
  - questo è infatti la somma di ogni prodotto di valori corrispondenti, che è 1 dove entrambe le righe hanno 1 e 0 altrimenti
- Ad es., il numero di acquisti in comune tra il primo e il secondo utente è ...

In [29]:
purchases[0].dot(purchases[1])

0

### Matrice di Similarità

- Per ottenere i prodotti scalari tra tutte le righe, possiamo **moltiplicare la matrice per la sua trasposta**
  - si moltiplicano le righe della matrice originale per le colonne della trasposta, che corrispondono sempre alle righe dell'originale

In [30]:
common_purchases = purchases.dot(purchases.T)

- Otteniamo una matrice quadrata simmetrica, di ordine pari al numero di utenti

In [31]:
# la matrice è uguale alla sua trasposta ?
np.array_equal(common_purchases, common_purchases.T)

True

In [32]:
common_purchases.shape

(178, 178)

- Visualizziamo un estratto della matrice...

In [33]:
common_purchases[:10, :10]

array([[ 58,   0,   0,   1,   2,   2,   1,   1,   3,   1],
       [  0,  40,   0,   1,   0,   3,   0,   0,   0,   0],
       [  0,   0,  31,   1,   0,   2,   0,   0,   0,   0],
       [  1,   1,   1,  37,   0,   1,   1,   1,   1,   0],
       [  2,   0,   0,   0,  39,   0,   0,   1,   3,   0],
       [  2,   3,   2,   1,   0, -77,   1,   1,   4,   0],
       [  1,   0,   0,   1,   0,   1,  72,   3,   1,   0],
       [  1,   0,   0,   1,   1,   1,   3,  41,   0,   0],
       [  3,   0,   0,   1,   3,   4,   1,   0,  59,   0],
       [  1,   0,   0,   0,   0,   0,   0,   0,   0,  55]], dtype=int8)

- ...possiamo ad esempio notare che
  - tra i primi due utenti ci sono 4 oggetti acquistati in comune
  - il primo ed il quarto utente non hanno alcun acquisto in comune

### Diagonale della Matrice

- La diagonale della matrice creata contiene i prodotti scalari di ciascuna riga della matrice originale con se stessa
- Si può verificare che sono uguali alla somma delle colonne calcolata sopra

In [34]:
np.array_equal(common_purchases.diagonal(), users_purchases)

False

- Prima di procedere, **impostiamo a 0 tutti i valori sulla diagonale** per far sì che tra i simili di ciascun utente non sia incluso lui stesso

In [35]:
# fill_diagonal imposta tutti gli elementi della diagonale al valore dato
np.fill_diagonal(common_purchases, 0)
common_purchases[:5, :5]

array([[0, 0, 0, 1, 2],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [1, 1, 1, 0, 0],
       [2, 0, 0, 0, 0]], dtype=int8)

_**Quesito:** qual è il massimo numero di prodotti in comune tra due utenti?_

## Stimare il Potenziale Interesse nei Prodotti

- Vogliamo stimare **quanto ciascun utente sia potenzialmente interessato** in ciascun prodotto non ancora acquistato
- Possiamo stimarlo in base a quanto il prodotto sia stato acquistato da utenti simili
- Moltiplichiamo la matrice degli acquisti comuni (che indica la similarità tra utenti) per quella degli acquisti: (utenti x utenti) x (utenti x prodotti) = utenti x prodotti

In [36]:
interest = common_purchases.dot(purchases)

- La forma della matrice è pari a quella della matrice degli acquisti (utenti x prodotti)

In [37]:
interest.shape == purchases.shape

True

- Visualizziamo un estratto della matrice...

In [38]:
interest[0:10, 0:10]

array([[ 8,  6,  3,  1,  2, 14,  1,  3,  5,  1],
       [ 0,  1,  4,  0,  2, 11,  0,  0,  2,  0],
       [ 0,  1,  1,  0,  0,  2,  0,  0,  0,  1],
       [ 3,  4,  1,  1,  0,  8,  0,  1,  3,  0],
       [ 4,  2,  0,  0,  0,  1,  0,  0,  0,  2],
       [ 6, 22,  8,  1,  3, 51,  0,  3, 10,  9],
       [ 2,  7,  3,  0,  0,  7,  0,  1,  3,  0],
       [ 0,  2,  1,  1,  0,  9,  0,  1,  0,  1],
       [ 2,  6,  3,  2,  2, 18,  0,  0,  6,  1],
       [ 0,  0,  4,  0,  0,  1,  0,  1,  0,  2]], dtype=int8)

- Ad esempio si ritiene che il 10° utente sia molto interessato al 4° prodotto

### Scartare i Prodotti già Acquistati

- Nella matrice calcolata `interest`, sono "interessanti" anche i prodotti **già acquistati** da ciascun utente
- Prima di proseguire, **azzeriamo i loro valori** in modo da considerare solo prodotti nuovi
- Convertiamo la matrice degli acquisti iniziale in forma booleana, per ottenere una "maschera"
  - `astype` copia un array convertendone tutti i valori in un altro tipo
  - come in Python standard, gli 0 sono convertiti in `False` e gli altri valori in `True`

In [39]:
purchases_mask = purchases.astype(np.bool)

- Utilizziamo quindi la maschera per **selezionare solo i valori corrispondenti** nella matrice dell'interesse all'acquisto che **impostiamo a 0**

In [40]:
interest[purchases_mask] = 0

## Ottenere _N_ Suggerimenti di Acquisto per ogni Utente

- Da migliaia di prodotti nel catalogo, vogliamo suggerirne **un numero limitato ad ogni utente** massimizzando la probabilità di acquisto
- Fissiamo un numero _N_ di prodotti da suggerire...

In [41]:
N = 20

- ...vogliamo selezionare per ogni utente gli _N_ prodotti con "potenziale interesse" maggiore

### Suggerimenti Basati sulla Previsione degli Interessi 

- Possiamo ottenere suggerimenti migliori sfruttando l'informazione sull'interesse stimato degli utenti verso i prodotti ?
- Per verificarlo, **selezioniamo gli _N_ prodotti di maggiore interesse stimato per ciascun utente**
- Procediamo come segue:
  - assegniamo a ciascun prodotto un "ranking" in base all'interesse per ciascun utente
  - selezioniamo per ogni cliente gli _N_ prodotti col ranking migliore
- Una volta estratti i suggerimenti, ne valuteremo l'accuratezza basandoci sugli acquisti che gli utenti hanno fatto in seguito

### Estrarre l'ordine dei valori: il metodo `argsort`

- Il metodo `argsort` su un vettore restituisce un vettore con i suoi **indici** (da 0 a N-1) **ordinati** secondo i valori

In [42]:
# sia dato il vettore...
x = np.array([32, 8, 2, 4, 16, 64, 1])
# ...il risultato di argsort è...
x.argsort()

array([6, 2, 3, 1, 4, 0, 5])

- Significa che l'elemento minimo è quello di indice 6 (1), seguito da quello di indice 2 (2) e così via fino al massimo di indice 5 (64)
- Nel caso di una matrice, l'operazione è eseguita **lungo una dimensione** a scelta (riga per riga o colonna per colonna)

In [43]:
np.array([[32,  8,  2,  4, 16, 64,  1],
          [ 8, 16,  4, 64, 32,  1,  2]]).argsort(1)   # 1 = riga per riga

array([[6, 2, 3, 1, 4, 0, 5],
       [5, 6, 2, 0, 1, 4, 3]])

### Calcolare il Ranking dei Prodotti

- Applicando l'operazione `argsort` due volte otteniamo il "ranking" dei dati, ovvero l'indice che ogni elemento avrebbe nell'array ordinato
  - Normalmente otteniamo quindi un array che associa 0 all'elemento più basso, 1 al secondo e così via

In [44]:
np.array([32, 8, 2, 4, 16, 64, 1]).argsort().argsort()

array([5, 3, 1, 2, 4, 6, 0])

- D'altra parte, invertendo l'array a cui è applicata l'operazione, otteniamo un array che associa 0 a partire dal valore più alto
- Applichiamo queste operazioni riga per riga all'array `interest`

In [45]:
interest_ranking = (-interest).argsort(1).argsort(1)

### Selezionare i Prodotti da Suggerire

- Abbiamo così ottenuto un array dove per ogni utente (riga) abbiamo i prodotti numerati univocamente da 0 a _N_-1 in ordine di interesse
- Da questo array possiamo quindi selezionare i **valori minori di _N_** per ottenere gli **_N_ prodotti di maggiore interesse** per ciascun utente
  - otteniamo una matrice di valori bool (True/False), che convertiamo in numerica con `astype`

In [46]:
suggestions = (interest_ranking < N).astype(np.int16)

- Questa è la **matrice dei prodotti suggeriti**, che associa ad ogni utente gli _N_ prodotti a cui è potenzialmente più interessato

## Accuratezza dei Suggerimenti di Acquisto

- Come valutare se i suggerimenti ottenuti in questo modo siano azzeccati ?
- Una possibilità consiste nel verificare **se gli oggetti suggeriti siano stati effettivamente acquistati** in un successivo momento
- Nel file `estore-purchases-2014.npy.gz` è fornita una seconda matrice degli acquisti aggiornata
  - righe e colonne sono corrispondenti agli stessi utenti e prodotti della prima matrice
  - i valori sono basati invece su tutti gli acquisti, anche quelli effettuati dopo il 2000
- Possiamo quindi confrontare i prodotti suggeriti con questa nuova matrice

In [47]:
purchases_updated = load_matrix("https://git.io/vx81t")

NameError: name 'load_matrix' is not defined

- Verifichiamo che la forma coincida con quella della precedente

In [11]:
purchases_updated.shape == purchases.shape

True

### Selezionare solo i Nuovi Acquisti

- La nuova matrice riporta **tutti** gli acquisti, compresi quelli già indicati nella matrice precedente
- Vogliamo una matrice in cui siano riportati solo gli acquisti successivi all'analisi svolta sopra
- Possiamo ottenerla con una semplice differenza tra le due matrici

In [12]:
new_purchases = purchases_updated - purchases

- Quanti nuovi acquisti ha fatto mediamente ogni utente ?


In [13]:
mean_new_purchases = new_purchases.sum() / n_users
mean_new_purchases

32.98314606741573

In [33]:
np.median(new_purchases.sum(1)) # mediana

6.5

### Quali Nuovi Acquisti sono stati Suggeriti ?

- Abbiamo ora la matrice `suggestions` con gli acquisti _suggeriti_ e quella `new_purchases` con i nuovi acquisti _effettivi_
- Da queste possiamo individuare quali sono i suggerimenti **validi**, quelli a cui dopo l'analisi è corrisposto un acquisto
- Sono in pratica l'intersezione tra i due insiemi, che otteniamo moltiplicando le matrici elemento per elemento
  - ciascun valore della nuova matrice sarà 1 solo dove **entrambi** i valori delle due esistenti è 1

In [34]:
hits = suggestions * new_purchases

### Quanti Clienti hanno Ricevuto almeno un Suggerimento Valido ?

- Possiamo usare la funzione `max` per vedere in quali righe di `hits` sia presente almeno un 1, corrispondente ad un suggerimento valido

In [35]:
satisfied_users = hits.max(1)

- Quanti sono gli utenti che hanno ricevuto un suggerimento valido ?

In [36]:
satisfied_users.sum()

61

- ...e quanti sono in percentuale sul totale

In [37]:
satisfied_users.sum() / n_users

0.34269662921348315

- Abbiamo quindi previsto per il **34,3%** degli utenti almeno un prodotto che poi hanno acquistato
- ma quanto è buono questo risultato ?

### Confronto con una Selezione Casuale di Prodotti

- Per verificare il valore (o bontà) del risultato, misuriamo l'accuratezza ottenibile **suggerendo _N_ prodotti a caso** a ciascun utente
- Per estrarre dei suggerimenti casuali, definiamo una matrice con valori di interesse a caso tra 0 e 1 ...
  - il modulo `random` contiene le funzionalità relative alla generazione di array con valori casuali secondo varie distribuzioni
  - la funzione `seed` imposta il seed per la generazione di numeri casuali, in modo da rendere riproducibile il risultato
  - la funzione `random` genera un array di valori casuali con distribuzione uniforme in [0, 1) con forma data

In [38]:
np.random.seed(5)
random_interest = np.random.random(interest.shape)
random_interest[:5, :5]

array([[0.22199317, 0.87073231, 0.20671916, 0.91861091, 0.48841119],
       [0.16621773, 0.4949298 , 0.81012877, 0.78034263, 0.63087014],
       [0.36680512, 0.54382329, 0.07050355, 0.21437738, 0.78046745],
       [0.19330102, 0.67788327, 0.04413307, 0.06011549, 0.56624796],
       [0.34956196, 0.84851566, 0.08981507, 0.21130447, 0.84895548]])

- Come prima, impostiamo a 0 i valori per prodotti già acquistati...

In [39]:
random_interest[purchases_mask] = 0

- ...e selezioniamo gli _N_ prodotti di maggiore interesse per ogni utente

In [40]:
random_interest_ranking = (-random_interest).argsort(1).argsort(1)
random_suggestions = (random_interest_ranking < N).astype(np.int16)

- Rieseguiamo infine il confronto con i prodotti effettivamente acquistati

In [41]:
random_hits = random_suggestions * new_purchases
random_satisfied_users = random_hits.max(1)

In [42]:
random_satisfied_users.sum()

22

In [43]:
random_satisfied_users.sum() / n_users

0.12359550561797752

## Suggerimenti Casuali: Qual è l'Accuratezza Teorica ?
- il risultato del 12% appena calcolato è frutto di un singolo esperimento e cambia cambiando i suggerimenti casuali proposti ad ogni utente
 - basta cambiare il seed, ad es. `np.random.seed(2)` produce un'accuratezza del 15%   
- qual è allora il risultato corretto indipendente dal singolo esperimento ?   
- definiamo il problema utilizzando il calcolo delle probabilità 
 - qual è la probabilità che un prodotto scelto a caso sia di interesse per un utente ?
 - con $p$ prodotti in catalogo non ancora acquistati ed $i$ prodotti di interesse per ogni utente, allora è 
   - 1 - la prob. di non fornire nessun prodotto di interesse
 - considerando suggerimenti di $n$ prodotti, la prob. di NON fornire prodotti di interesse è 
 
$$\frac{\binom{p - i}{n}}{\binom{p}{n}}$$

- Nel nostro caso, assumiamo il numero di prodotti suggeribili per utente pari alla differenza tra quelli noti e quelli in media già acquistati

In [44]:
p = n_items - mean_purchases
p

3329.6011235955057

- Ipotizziamo che il numero di prodotti d'interesse per ciascun utente sia _i_ = 22
- In base a questo, la probabilità di azzeccare casualmente almeno un prodotto di interesse suggerendone 20 è ...

In [45]:
# importiamo la funzione che calcola il coefficiente binomiale
from scipy.special import binom
1 - binom(p-22 ,N) / binom(p ,N)

0.1245064688633315

## Conclusioni ##

- Proponendo suggerimenti di acquisto a caso, azzecchiamo gli acquisti futuri (periodo di test) per circa il **12%** degli utenti
 - identico di fatto al risultato teorico probabilistico del **12%** con $i=22$
- il metodo di recommendation esposto, basato sul semplice prodotto della matrice _utenti x prodotti_, è circa 3 volte più efficace di quello casuale
- ma come stimiamo il numero di prodotti di interesse, i.e. i = 22 ?
 - dal numero di acquisti che ipotizziamo facciano mediamente i clienti sulla base di quelli fatti in passato a parità di estensione temporale
 - essendo questo un esperimento volto a misurare quanto il risultato sia migliore di quello con suggerimenti casuali, $i$ è il num. di acquisti noti nel periodo di test  
 - tuttavia il numero medio di acquisti nel test è 33, ma con $i=33$ la prob. diventa 18%, perchè è più alta rispetto all'esperimento ?

## Esercizio: Calcoliamo la Probabilità Utente per Utente
- la probabilità teorica di suggerire casualmente prodotti validi è più alta dell'accuratezza ottenuta dal medesimo esperimento con $i=33$: 18% vs 12%, perchè ?
  - usare il semplice num. medio di acquisti implica assumere che questa probabilità aumenti linearmente rispetto ad $i$, ma ciò non è vero 
   - e.g. un utente col doppio degli acquisti di un altro, non ha una probabilità doppia di ricevere almeno un suggerimento di acquisto valido a parità di num. di suggerimenti
- **Esercizio**: calcolare questa probabilità utente per utente con $i$ diverso per ogni utente
 - la prob. teorica giusta è la media delle prob. calcolate per ciascun utente
 - il risultato è circa il 13%
 - Per verificarne la correttezza sperimentalmente, eseguire almeno un centinaio di simulazioni di suggerimenti di acquisti casuali e farne la media
 - per la legge dei grandi numeri, all'aumentare delle simulazioni quest'ultima media si avvicinerà sempre di più a quello esatto.

In [46]:
users_new_purchases = new_purchases.sum(1)
hit_probs = np.zeros((n_users, ))
for i in range(n_users):
    hit_probs[i] = 1 - binom(n_items-users_purchases[i]-users_new_purchases[i], N) / binom(n_items-users_purchases[i], N)
hit_probs.mean()

0.13028989825011686